In [1]:
import modin.pandas as pd
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
import seaborn as sns
import ipywidgets
import variables
import functions

In [2]:
df = pd.read_csv(variables.file, encoding='ISO-8859-1')
df

,Periodo,Entidad_Federativa,Sexo,Grupo_edad,Nivel_ingreso,Numero_personas
0,20050301,Aguascalientes,Hombre,15 A 24 A¥OS,Menos de 1 s.m.,4284
1,20050301,Aguascalientes,Hombre,15 A 24 A¥OS,1 salario m¡nimo,179
2,20050301,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 2 hasta 3 s.m.,10503
3,20050301,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 3 hasta 5 s.m.,16803
4,20050301,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 5 hasta 10 s.m.,1955
...,...,...,...,...,...,...
225721,20190601,Zacatecas,Mujer,45 A 64 A¥OS,No recibe ingresos,136
225722,20190601,Zacatecas,Mujer,45 A 64 A¥OS,No especificado,561
225723,20190601,Zacatecas,Mujer,65 A¥OS Y MAS,Menos de 1 s.m.,1521
225724,20190601,Zacatecas,Mujer,65 A¥OS Y MAS,M s de 3 hasta 5 s.m.,1281


In [3]:
df.insert(0, 'year', 0, True)
df.insert(1, 'month', 0, True)
df.insert(2, 'day', 0, True)
df['year'] = df['Periodo'].astype(str).str.slice(start=0, stop=4)
df['month'] = df['Periodo'].astype(str).str.slice(start=4, stop=6)
df['day'] = df['Periodo'].astype(str).str.slice(start=6, stop=8)
df.insert(0, 'date', df['year'] + '-' + df['month'] + '-' + df['day'])
df.drop(columns='Periodo', inplace=True)

In [4]:
df.rename(columns={'Entidad_Federativa': 'state', 'Sexo': 'sex', 'Grupo_edad': 'age_range', 'Nivel_ingreso': 'level', 'Numero_personas': 'num_people'}, inplace=True)
df = df.dropna();
df

,date,year,month,day,state,sex,age_range,level,num_people
0,2005-03-01,2005,03,01,Aguascalientes,Hombre,15 A 24 A¥OS,Menos de 1 s.m.,4284
1,2005-03-01,2005,03,01,Aguascalientes,Hombre,15 A 24 A¥OS,1 salario m¡nimo,179
2,2005-03-01,2005,03,01,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 2 hasta 3 s.m.,10503
3,2005-03-01,2005,03,01,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 3 hasta 5 s.m.,16803
4,2005-03-01,2005,03,01,Aguascalientes,Hombre,15 A 24 A¥OS,M s de 5 hasta 10 s.m.,1955
...,...,...,...,...,...,...,...,...,...
225721,2019-06-01,2019,06,01,Zacatecas,Mujer,45 A 64 A¥OS,No recibe ingresos,136
225722,2019-06-01,2019,06,01,Zacatecas,Mujer,45 A 64 A¥OS,No especificado,561
225723,2019-06-01,2019,06,01,Zacatecas,Mujer,65 A¥OS Y MAS,Menos de 1 s.m.,1521
225724,2019-06-01,2019,06,01,Zacatecas,Mujer,65 A¥OS Y MAS,M s de 3 hasta 5 s.m.,1281


In [5]:
group_state_year_people = df.groupby(['state', 'year']).sum()

In [6]:
group_state_year_people = group_state_year_people.reset_index()
group_state_year_people.rename(columns={'num_people': 'total_people'}, inplace=True)
group_state_year_total_people = group_state_year_people
group_state_year_total_people

,state,year,total_people
0,Aguascalientes,2005,1001646
1,Aguascalientes,2006,1045773
2,Aguascalientes,2007,1074998
3,Aguascalientes,2008,1040027
4,Aguascalientes,2009,1018094
...,...,...,...
490,Zacatecas,2015,1636378
491,Zacatecas,2016,1646459
492,Zacatecas,2017,1695298
493,Zacatecas,2018,1687911


In [7]:
df2 = df.groupby(['year', 'state']).sum()

In [8]:
df2.reset_index(inplace=True)
df2.rename(columns={'num_people': 'total_people_by_state_year'}, inplace=True)

In [9]:
df3 = df.groupby(['year','state', 'level']).sum()

In [10]:
df3.reset_index(inplace=True)
df3.rename(columns={'num_people': 'total_people_by_state_and_level'}, inplace=True)
df3['total_man_by_state_and_level'] = 0
df3['total_woman_by_state_and_level'] = 0

df3['total_people_by_state_year'] = 0
df3['total_man_by_state_year'] = 0
df3['total_woman_by_state_year'] = 0

df3['man_percentage_by_state_and_level'] = 0
df3['woman_percentage_by_state_and_level'] = 0

In [11]:
df3

,year,state,level,total_people_by_state_and_level,total_man_by_state_and_level,total_woman_by_state_and_level,total_people_by_state_year,total_man_by_state_year,total_woman_by_state_year,man_percentage_by_state_and_level,woman_percentage_by_state_and_level
0,2005,Aguascalientes,1 salario m¡nimo,3157,0,0,0,0,0,0,0
1,2005,Aguascalientes,Menos de 1 s.m.,118401,0,0,0,0,0,0,0
2,2005,Aguascalientes,M s de 1 hasta 2 s.m.,5092,0,0,0,0,0,0,0
3,2005,Aguascalientes,M s de 10 s.m.,257632,0,0,0,0,0,0,0
4,2005,Aguascalientes,M s de 2 hasta 3 s.m.,140654,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4450,2019,Zacatecas,M s de 2 hasta 3 s.m.,125065,0,0,0,0,0,0,0
4451,2019,Zacatecas,M s de 3 hasta 5 s.m.,160188,0,0,0,0,0,0,0
4452,2019,Zacatecas,M s de 5 hasta 10 s.m.,31272,0,0,0,0,0,0,0
4453,2019,Zacatecas,No especificado,5493,0,0,0,0,0,0,0


In [12]:
df4 = df.groupby(['year','state', 'level', 'sex']).sum()

In [13]:
df4.reset_index(inplace=True)

In [14]:
df4

,year,state,level,sex,num_people
0,2005,Aguascalientes,1 salario m¡nimo,Hombre,3157
1,2005,Aguascalientes,Menos de 1 s.m.,Hombre,111212
2,2005,Aguascalientes,Menos de 1 s.m.,Mujer,7189
3,2005,Aguascalientes,M s de 1 hasta 2 s.m.,Hombre,3921
4,2005,Aguascalientes,M s de 1 hasta 2 s.m.,Mujer,1171
...,...,...,...,...,...
8810,2019,Zacatecas,M s de 5 hasta 10 s.m.,Mujer,5011
8811,2019,Zacatecas,No especificado,Hombre,3914
8812,2019,Zacatecas,No especificado,Mujer,1579
8813,2019,Zacatecas,No recibe ingresos,Hombre,16920


In [15]:
#for index, row in df3.iterrows():
#    for index2, row2 in df4.iterrows():
#        
#        if row['year'] == row2['year'] and row['state'] == row2['state'] and row['level'] == row2['level']:
#            
#            r = row2['num_people']
#            
#            if row2['sex'] == 'Hombre':                            
#                df3.at[index, 'total_man_by_state_and_level_year'] = r
#                
#            if row2['sex'] == 'Mujer':
#                df3.at[index, 'total_woman_by_state_and_level_year'] = r

In [16]:
# df3.to_csv('firstPart.csv')

In [17]:
df3 = pd.read_csv('firstPart.csv')
df3 = df3.astype({"year": int})
df3 = df3.astype({"total_people_by_state_and_level": int})
df3 = df3.astype({"total_man_by_state_and_level": int})
df3 = df3.astype({"total_woman_by_state_and_level": int})
df3 = df3.astype({"total_people_by_state": int})
df3 = df3.astype({"total_man_by_state": int})
df3 = df3.astype({"total_woman_by_state": int})

In [18]:
df3.rename(columns={'total_people_by_state':'total_people_by_state_and_year','total_man_by_state':'total_man_by_state_and_year','total_woman_by_state':'total_woman_by_state_and_year'}, inplace=True)

In [19]:
df3['man_percentage_by_state_and_level'] = 0
df3['woman_percentage_by_state_and_level'] = 0
df3['rate_level'] = 0
df3

,year,state,level,total_people_by_state_and_level,total_man_by_state_and_level,total_woman_by_state_and_level,total_people_by_state_and_year,total_man_by_state_and_year,total_woman_by_state_and_year,man_percentage_by_state_and_level,woman_percentage_by_state_and_level,rate_level
0,2005,Aguascalientes,1 salario m¡nimo,3157,3157,0,0,0,0,0,0,0
1,2005,Aguascalientes,Menos de 1 s.m.,118401,111212,7189,0,0,0,0,0,0
2,2005,Aguascalientes,M s de 1 hasta 2 s.m.,5092,3921,1171,0,0,0,0,0,0
3,2005,Aguascalientes,M s de 10 s.m.,257632,124638,132994,0,0,0,0,0,0
4,2005,Aguascalientes,M s de 2 hasta 3 s.m.,140654,135145,5509,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4450,2019,Zacatecas,M s de 2 hasta 3 s.m.,125065,123622,1443,0,0,0,0,0,0
4451,2019,Zacatecas,M s de 3 hasta 5 s.m.,160188,93125,67063,0,0,0,0,0,0
4452,2019,Zacatecas,M s de 5 hasta 10 s.m.,31272,26261,5011,0,0,0,0,0,0
4453,2019,Zacatecas,No especificado,5493,3914,1579,0,0,0,0,0,0


In [20]:
group_by_state = df.groupby(['state', 'year']).sum()
group_by_state.reset_index(inplace=True)
group_by_state = group_by_state.astype({"year": int})

In [21]:
group_by_state

,state,year,num_people
0,Aguascalientes,2005,1001646
1,Aguascalientes,2006,1045773
2,Aguascalientes,2007,1074998
3,Aguascalientes,2008,1040027
4,Aguascalientes,2009,1018094
...,...,...,...
490,Zacatecas,2015,1636378
491,Zacatecas,2016,1646459
492,Zacatecas,2017,1695298
493,Zacatecas,2018,1687911


In [ ]:
for index, row in group_by_state.iterrows():
    for index2, row2 in df3.iterrows():
        if row['state'] == row2['state'] and row['year'] == row2['year']:
            r = row['num_people']
            df3.at[index2, 'total_people_by_state_and_year'] = r
            continue

In [ ]:
df3

In [ ]:
group_by_state_and_sex = df.groupby(['state', 'sex', 'year']).sum()
group_by_state_and_sex.reset_index(inplace=True)
group_by_state_and_sex = group_by_state_and_sex.astype({"year": int})
group_by_state_and_sex = group_by_state_and_sex.astype({"num_people": int})
group_by_state_and_sex

In [ ]:
for index, row in df3.iterrows():
    for index2, row2 in group_by_state_and_sex.iterrows():
        
        if row['state'] == row2['state'] and row['year'] == row2['year']:
            
            r = row2['num_people']
            
            if row2['sex'] == 'Hombre':                            
                df3.at[index, 'total_man_by_state_and_year'] = r
                
            if row2['sex'] == 'Mujer':
                df3.at[index, 'total_woman_by_state_and_year'] = r
            
            continue

In [ ]:
df3.to_csv('secondPart.csv')

In [ ]:
# add percentage
df3['man_percentage_by_state_and_level'] = (df3['total_man_by_state_and_level'] / df3['total_people_by_state_and_level']) * 100
df3['woman_percentage_by_state_and_level'] = (df3['total_woman_by_state_and_level'] / df3['total_people_by_state_and_level']) * 100

In [ ]:
df3['man_percentage_by_state_and_level'] = df3['man_percentage_by_state_and_level'].apply(lambda x: round(x, 3))
df3['woman_percentage_by_state_and_level'] = df3['woman_percentage_by_state_and_level'].apply(lambda x: round(x, 3))

In [ ]:
state_filter = df['state'] == 'Aguascalientes'
df3.where(state_filter).dropna()

In [ ]:
df3.to_csv('thirdPart.csv')